In [11]:
import pandas as pd
import sqlite3

## Подключение через `sqlite3`

In [12]:
data = sqlite3.connect("../data/checking-logs.sqlite")

## Создать таблицу `datamart` с колонками: 
- `uid`
- `labname`
- `first_commit_ts`
- `first_view_ts`

## first_commit_ts — это timestamp из checker для первого коммита пользователя по конкретной лабораторной.

## first_view_ts — время первого посещения новостной ленты (pageviews).

## Фильтры:

- `status = 'ready'`

- `numTrials = 1`

- `labname` в списке выше

- Только пользователи `user_*`

---

- `CREATE TABLE IF NOT EXISTS datamart AS` -> Создает таблицу `datamart` если она еще не существует. `AS` Указатель на то что таблица будет создана на основе следующих запросов

- `SELECT checker.uid, checker.labname, checker.timestamp AS first_commit_ts, pageviews.datetime AS first_view_ts`

    - Выбирает 4 столбца с разных таблиц и 2 из них переименовывает с помощью `AS`

- `FROM checker LEFT JOIN pageviews ON pageviews.uid = checker.uid`

    - Левая основная таблица `checker` будет добавлена (внедрена) в правую таблицу `pageviews` с помощью `LEFT JOIN`

        - `LEFT JOIN`
            
            - Все строки из левой таблицы `checker` включаются в результат, есть ли соответствующие строки в правой таблице `pageviews`
            
            - Для строк из `checker` у которых нет соответствия в `pageviews` значения столбцов из `pageviews` будут `NULL` в результирующем наборе данных

            - Это сохранит все записи из `checker`, даже если у некоторых нет записей в `pageviews`

- `WHERE checker.uid LIKE "user_%"` -> условия по которой 2 таблицы будут соединены.

    - `AND status = 'ready'`

    - `AND numTrials = 1`

    - `AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')`

    - `AND (pageviews.datetime = (SELECT MIN(datetime) FROM pageviews WHERE checker.uid = pageviews.uid) OR pageviews.datetime IS NULL)`

        - `(pageviews.datetime = (SELECT MIN(datetime) FROM pageviews WHERE checker.uid = pageviews.uid)`

            - тут спрашивается - Если `datetime` с `pageviews` равна минимальному значению `datetime` с `pageviews` и при этом `uid` 2 таблиц равны

            - либо - `datetime` с `pageviews` равна `NULL`

---

- `data.execute(sql_request)` - `execute` -> отправляет запрос в БД

In [13]:
sql_request = """
CREATE TABLE IF NOT EXISTS datamart AS

SELECT checker.uid, checker.labname, checker.timestamp AS first_commit_ts, pageviews.datetime AS first_view_ts

FROM checker LEFT JOIN pageviews ON pageviews.uid = checker.uid

WHERE checker.uid LIKE 'user_%'

AND status = 'ready'

AND numTrials = 1

AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')

AND (pageviews.datetime = (SELECT MIN(datetime) FROM pageviews WHERE checker.uid = pageviews.uid) OR pageviews.datetime IS NULL)
"""

data.execute(sql_request)

## Преобразуйте временные поля в datetime64[ns]

---

- `pd.read_sql()` - 3 параметра

    - 1. `SQL` запрос

    - 2. куда этот запрос отправиться

    - 3. `parse_dates=['first_commit_ts', 'first_view_ts']`

        - yказывает *pandas* автоматически преобразовать столбцы `first_commit_ts` и `first_view_ts` в тип `datetime64[ns]` при чтении

In [14]:
datamart = pd.read_sql('SELECT * FROM datamart', data, parse_dates=['first_commit_ts', 'first_view_ts'])

datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


## Создайте DataFrame test (с пользователями, у которых есть first_view_ts) и control (у которых его нет).

## В control замените пропуски средним временем первого просмотра из test.

---

- создаем 2 датафрейма из `datamart`

- 1. `test` - берет в себя строки в которых значения `first_view_ts` не `NULL`

- 2. `control` - берет в себя строки в которых  `first_view_ts` == `NUL`

In [15]:
test = datamart[datamart["first_view_ts"].notnull()]

control = datamart[datamart["first_view_ts"].isnull()]

In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [17]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


## Сохраните обе таблицы в БД

In [21]:
# test.to_sql("test", data)
# control.to_sql("control", data)

pd.read_sql("SELECT * FROM test LIMIT 10", data)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## Закрываем соединение с БД

In [22]:
data.close()